# Module 2 - Problem 7: Balancing an redox reaction

Indetify the substances that is oxidized from the reduced of a reaction with two reactives and two products.
Moreover, the oxidation number of the same atom in the both reactives or products should be equal. This program
can also calculate the oxidation number of one atom in a molecule

    * Input reactants and products with their charges
    * Input oxidation numbers
        * Calculate of oxidation number
    * Electronic movement
    * Classify the substance in reducing and oxidizing agents

Notes:

    * Oxidation number: is a virtual charge belong to an atom whether every bonds are ionic
        * An atom of free element has an oxidation number equal to 0, example, each atom in Cl2
        * An monoatomic ion has an oxidation number equal to the charge
        * Alkali metals in a molecule always have an oxidation number equal to 1
        * Alkali earth metals in a molecule always have an oxidation number equal to 2
        * Fluorine in compounds has an oxidation number equal to -1
        * Hydrogen in compoinds has an oxidation number equal to 1 except in molecules with
          metals because in these substances its oxidation number equal to -1, example, NaH
        * Oxygen atom has an oxidation number equal to -2 except in the peroxide because 
          in this substance its oxidation number is equal to -1
        * Other halogens have an oxidation number equal to -1 but in molecules with oxygen
          or fluorine its oxidation number change, example, ClO4- the Cl is 7

In [4]:
from sys import path

path.append("../../src/m1")

import numpy as np
from atomic_mass_percentage import split_symbols_amount, molecular_mass

In [24]:
def input_substance(num_react: int, num_prod: int) -> [list[str], list[int]]:

    molecules: list[str] = []
    charges: list[int] = []
    for r in range(num_react):
        molecules.append(input(f"what is the {r+1} reactive?"))
        charges.append(int(input(f"what is the charge of the {molecules[r]}?")))

    for p in range(num_prod):
        molecules.append(input(f"what is the {p+1} product?"))
        charges.append(int(input(f"what is the charge of the {molecules[num_react+p]}?")))

    return molecules, charges

In [6]:
def molecule_composition(molecule: list) -> dict[str : int]:
    for m in molecule:
        yield split_symbols_amount(m)

In [31]:
def calculate_oxidation_number(
    molecule_composition: dict[str:int], oxidation_number: dict[str:int], charge: int
) -> dict[str:int]:
    for atom, no in oxidation_number.items():
        if no == 999:
            num_at: int = molecule_composition[atom]
            temp: int = charge - sum(
                [
                    on * na
                    for na, on in zip(
                        molecule_composition.values(), oxidation_number.values()
                    )
                    if on != 999
                ]
            )
            temp = temp // num_at
            oxidation_number[atom] = temp

    return oxidation_number

In [9]:
def input_oxidation_number(
    molecules_composition: list[dict[str : int]], molecules: list[str], charges: list[int]
) -> list[dict[str : int]]:
    
    for count, mol in enumerate(molecules_composition):
        oxidation_number: dict[str:int] = {}
        for atom in mol.keys():
            oxidation_number[atom] = int(
                input(f"what is the oxidation number of atom {atom} in {molecules[count]}? ")
            )

        if 999 in oxidation_number.values():
            oxidation_number = calculate_oxidation_number(
                mol, oxidation_number, charges[count]
            )
            
        yield oxidation_number

In [67]:
def redox(react: dict[str : int], prod: dict[str : int], r: str, p: str) -> dict[str : int]:
    
    oxidation_numbers: dict[str : int] = {}
    for atr, onr in react.items():
        if atr in prod.keys():
            if prod[atr] > onr:
                print(
                    f"The atom {atr} oxidizes when pass from {r} to {p} ({onr}-->{prod[atr]})"
                )
                oxidation_numbers[atr] = prod[atr] - onr
                oxidation_numbers["Oxidize"] = True
            elif prod[atr] == onr:
                print(
                    f"The atom {atr} doesn't suffer oxide-reduction reaction  when pass from {r} to {p}"
                )
            else:
                print(
                    f"The atom {atr} reduces when pass from {r} to {p} ({onr}-->{prod[atr]})"
                )
                oxidation_numbers[atr] = onr - prod[atr]
                oxidation_numbers["Oxidize"] = False
    return oxidation_numbers

In [20]:
def oxide_reduction(
    no_react_prod: list[dict[str : int]], molecules: list[str], num_react: int, num_prod: int
) -> dict[str:int] or None:
    
    for r in range(num_react):
        react: dict[str : int] = no_react_prod[r]
        for p in range(num_prod):
            prod: dict[str : int] = no_react_prod[p+num_react]
            
            redox_numbers = redox(react,prod,molecules[r],molecules[p+num_react])

            if len(redox_numbers) > 0:
                yield redox_numbers

In [49]:
def matrix_amount_charge(atoms: set[str], comp_react_prod: list[dict[str : int]], charges: list[int], num_r: int) -> np.array:
# Note: The reactive coefficient are also positive because negative values make mistake in some reaction, for example
# F2+2 + Cr2O7+2 + H+1 --> Cr+3 + H2O + Fe+3
    matrix: list[list[int]] = []
    for a in atoms:            
        temp: list[int] = []
        for count, molecule in enumerate(comp_react_prod):
            if a in molecule.keys(): 
                temp.append(molecule[a])
            else:
                temp.append(0)
        matrix.append(temp)
    matrix.append(charges)

    return np.array(matrix,dtype=np.float_)

In [12]:
def heaviest(molecules: list[dict[str : int]]) -> int:

    heaviest_index: int = 0
    heaviest: float = 0.
    for count, mol in enumerate(molecules):
        mass_mol: float = molecular_mass(mol)
        if mass_mol > heaviest:
            heaviest = mass_mol
            heaviest_index = count

    return heaviest_index

In [13]:
def add_row_col(matrix: np.array, heaviest: int, nrow: int, ncol: int) -> np.array:

    if nrow > ncol:
        column_add: np.array = np.array([0 if i < (nrow - 1) else 1 for i in range(nrow)], dtype=np.float_)
        return np.column_stack((matrix, column_add))
    else:
        row_add: np.array = np.array([1 if i == heaviest else 0 for i in range(ncol)], dtype=np.float_)
        return np.r_[matrix,[row_add]]

In [14]:
def balance(mat_comp_mol: np.array, coef_result: np.array) -> np.array:

    return np.linalg.solve(mat_comp_mol, coef_result)

In [25]:
num_react: int = int(input("how many reactives are there?"))
num_prod: int = int(input("how many products are there?"))
molecules, charges = input_substance(num_react, num_prod)

In [9]:
reactive_a = "HNO3"
reactive_b = "Sn"
product_a = "H2SnO3"
product_b = "NO"
r_charge_a = r_charge_b = p_charge_a = p_charge_b = 0

In [30]:
comp_react_prod: list[dict[str : int]] = [comp for comp in molecule_composition(molecules)]

print(
    "if you unknown the oxidation number, please input 999 and the program calculate it (only maximum one atom by molecule)"
)

print("\n","*"*60,"\n")

num_oxid_atoms_react_prod: list = [no for no in input_oxidation_number(comp_react_prod, molecules, charges)]

print("\n","*"*60,"\n")

for mol, num_oxid in zip(molecules,num_oxid_atoms_react_prod):
    print(f"Number of oxidation of atoms in the {mol}: {num_oxid}")


if you unknown the oxidation number, please input 999 and the program calculate it (only maximum one atom by molecule)

 ************************************************************ 


 ************************************************************ 

Number of oxidation of atoms in the Fe: {'Fe': 2}
Number of oxidation of atoms in the Cr2O7: {'Cr': 6.0, 'O': -2}
Number of oxidation of atoms in the H: {'H': 1}
Number of oxidation of atoms in the Cr: {'Cr': 3}
Number of oxidation of atoms in the H2O: {'H': 1, 'O': -2}
Number of oxidation of atoms in the Fe: {'Fe': 3}


In [68]:
print("\n","*"*60,"\n")

nums_oxid: list[dict[str : int]] = [diff_no for diff_no in oxide_reduction(num_oxid_atoms_react_prod,molecules,num_react,num_prod)]

print("\n","*"*60,"\n")

print("Change the oxidation number of atoms: ")
print(nums_oxid)


 ************************************************************ 

The atom Fe oxidizes when pass from Fe to Fe (2-->3)
The atom Cr reduces when pass from Cr2O7 to Cr (6.0-->3)
The atom O doesn't suffer oxide-reduction reaction  when pass from Cr2O7 to H2O
The atom H doesn't suffer oxide-reduction reaction  when pass from H to H2O

 ************************************************************ 

Change the oxidation number of atoms: 
[{'Fe': 1, 'Oxidize': True}, {'Cr': 3.0, 'Oxidize': False}]


In [50]:
atoms: set[str] = {atom for molecule in comp_react_prod[:num_react] for atom in molecule.keys()}
matrix_amount_atoms = matrix_amount_charge(atoms, comp_react_prod, charges, num_react)

In [51]:
row, col = matrix_amount_atoms.shape
if row == col:
    print(f"The matrix with amount of the atoms is square:  {row} x {col}")
    m_square_amount_atoms = matrix_amount_atoms
    coef_result: np.array = np.array([0 for i in range(row)], dtype=np.float_)
else:
    print(f"The matrix with amount of the atoms is not square:  {row} x {col}")
    message: str = 'col'
    n: int = row
    if col > row:
        message = 'row'
        n = col
    
    print(f"Then, is add one {message} in the last")
    heaviest_molecule = heaviest(comp_react_prod)
    
    m_square_amount_atoms = add_row_col(matrix_amount_atoms, heaviest_molecule, row, col)

    coef_result: np.array = np.array([0 if i < (n-1) else 1 for i in range(n)], dtype=np.float_)

The matrix with amount of the atoms is not square:  5 x 6
Then, is add one row in the last


In [53]:
balance_coeff = balance(m_square_amount_atoms,coef_result)
print(f"Balance coeffecients from reactives to products: {balance_coeff}")

Balance coeffecients from reactives to products: [ 6.  1. 14. -2. -7. -6.]


In [69]:
for count, molecule in enumerate(molecules):
    bc: str = ''
    if abs(balance_coeff[count]) > 1:
        bc = str(abs(round(balance_coeff[count],1)))
    print(bc + " " + molecule, end=" ")
    if count < num_react - 1 or count >= num_react and count < num_react + num_prod - 1:
        print("+",end=" ")
    if count == num_react - 1:
        print("-->", end=" ")
print("\n")
for ox_red in nums_oxid:
    oxid_red: str = "oxidize"
    lost_gain: str = "lost"
    if not ox_red["Oxidize"]:
        oxid_red = "reduce"
        lost_gain = "gain"
    print(f"The atom {list(ox_red.keys())[0]} is {oxid_red}, {lost_gain} {list(ox_red.values())[0]} electrons")


6.0 Fe + 1.0 Cr2O7 + 14.0 H --> 2.0 Cr + 7.0 H2O + 6.0 Fe 

The atom Fe is oxidize, lost 1 electrons
The atom Cr is reduce, gain 3.0 electrons
